#  Flight Analytics 2024 — Analysis Notebook

**Autor:** Fabrice Martial  
**Erstellt:** 2025-10-14  

Dieses Notebook analysiert den Kaggle-Datensatz **Flight Data 2024**.  


**Inhalte:**
- Laden der Daten (SQL oder CSV)
- Datenqualität & Überblick
- KPIs (Verspätungen, Stornos, Flüge)
- Visualisierungen (monatlich, Airline, Routen)
- Export von Abbildungen




In [ ]:
# =========================
#  Konfiguration
# =========================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Matplotlib: Jede Grafik separat, keine Styles/Colors setzen
plt.rcParams.update({'figure.figsize': (9, 5)})

# Datenpfade (CSV-Fallback)
CSV_PATH_CLEAN = os.path.join('data', 'flight_data_2024_clean.csv')

# SQL-Parameter 
USE_SQL = True   
SQL_SERVER = r'DESKTOP-CFP338R'        
SQL_DATABASE = 'FlightAnalytics2024'
SQL_TABLE = 'dbo.flight_data_2024'  


SQL_TRUSTED = True
SQL_UID = ''
SQL_PWD = ''

print('Konfiguration geladen.')

Konfiguration geladen.


In [24]:
# =========================
#  Daten laden (SQL oder CSV)
# =========================
df = None

if USE_SQL:
    import pyodbc
    driver_candidates = [d for d in pyodbc.drivers() if 'SQL Server' in d]
    picked_driver = driver_candidates[-1] if driver_candidates else 'ODBC Driver 17 for SQL Server'
    if SQL_TRUSTED:
        conn_str = (
            f'DRIVER={{{picked_driver}}};'
            f'SERVER={SQL_SERVER};'
            f'DATABASE={SQL_DATABASE};'
            'Trusted_Connection=yes;'
            'TrustServerCertificate=yes;'
            'Encrypt=no;'
        )
    else:
        conn_str = (
            f'DRIVER={{{picked_driver}}};'
            f'SERVER={SQL_SERVER};'
            f'DATABASE={SQL_DATABASE};'
            f'UID={SQL_UID};PWD={SQL_PWD};'
            'TrustServerCertificate=yes;'
            'Encrypt=no;'
        )
    conn = pyodbc.connect(conn_str)
    query = f'SELECT * FROM {SQL_TABLE};'
    df = pd.read_sql(query, conn)
    conn.close()
else:
    df = pd.read_csv(CSV_PATH_CLEAN, parse_dates=['fl_date'])

# Typen nachziehen (falls CSV ohne Parse)
if 'fl_date' in df.columns and not np.issubdtype(df['fl_date'].dtype, np.datetime64):
    df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')

print(' Daten geladen:', df.shape)

C:\Users\fabri\AppData\Local\Temp\ipykernel_21268\4182767101.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


 Daten geladen: (10000, 35)


## 🔎 Datenqualität & Überblick

In [14]:
display(df.head(10))
print('\nSpalten:', list(df.columns))
print('\nNullwerte je Spalte (Top 10):')
print(df.isnull().sum().sort_values(ascending=False).head(10))

,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,4,18,4,2024-04-18,MQ,3535,DFW,"Dallas/Fort Worth, TX",Texas,...,False,151.0,144.0,119.0,835.0,0,0,0,0,0
1,2024,1,1,1,2024-01-01,AA,148,CLT,"Charlotte, NC",North Carolina,...,False,286.0,273.0,253.0,1773.0,0,0,0,0,0
2,2024,12,12,4,2024-12-12,9E,5440,CHA,"Chattanooga, TN",Tennessee,...,False,59.0,50.0,29.0,106.0,0,0,0,0,0
3,2024,4,8,1,2024-04-08,WN,1971,OMA,"Omaha, NE",Nebraska,...,False,180.0,177.0,163.0,1099.0,0,0,0,0,0
4,2024,2,16,5,2024-02-16,WN,862,BWI,"Baltimore, MD",Maryland,...,False,90.0,96.0,76.0,399.0,0,0,0,0,0
5,2024,4,15,1,2024-04-15,WN,2358,SAN,"San Diego, CA",California,...,False,75.0,68.0,51.0,304.0,0,0,0,0,0
6,2024,11,27,3,2024-11-27,G4,163,MLB,"Melbourne, FL",Florida,...,False,149.0,154.0,137.0,914.0,0,0,0,0,0
7,2024,8,31,6,2024-08-31,AS,1269,BNA,"Nashville, TN",Tennessee,...,False,300.0,278.0,254.0,1973.0,0,0,0,0,0
8,2024,4,19,5,2024-04-19,OO,3438,LAX,"Los Angeles, CA",California,...,False,79.0,83.0,49.0,308.0,0,0,0,0,0
9,2024,2,8,4,2024-02-08,OO,5654,SFO,"San Francisco, CA",California,...,False,75.0,60.0,40.0,238.0,0,0,0,0,0



Spalten: ['year', 'month', 'day_of_month', 'day_of_week', 'fl_date', 'op_unique_carrier', 'op_carrier_fl_num', 'origin', 'origin_city_name', 'origin_state_nm', 'dest', 'dest_city_name', 'dest_state_nm', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled', 'cancellation_code', 'diverted', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']

Nullwerte je Spalte (Top 10):
actual_elapsed_time    164
wheels_on              127
arr_time               127
wheels_off             120
dep_time               116
year                     0
month                    0
day_of_month             0
fl_date                  0
origin                   0
dtype: int64


##  KPIs

In [15]:
total_flights = len(df)
cancelled_flights = int((df['cancelled'] == 1).sum()) if 'cancelled' in df else np.nan
avg_arr_delay = float(df['arr_delay'].mean()) if 'arr_delay' in df else np.nan
avg_dep_delay = float(df['dep_delay'].mean()) if 'dep_delay' in df else np.nan

print(f'Total Flights: {total_flights:,}')
print(f'Cancelled Flights: {cancelled_flights:,}')
print(f'Average Arrival Delay (min): {avg_arr_delay:.2f}')
print(f'Average Departure Delay (min): {avg_dep_delay:.2f}')

Total Flights: 10,000
Cancelled Flights: 122
Average Arrival Delay (min): 7.42
Average Departure Delay (min): 12.85


##  Feature Engineering (Monat/Jahr)

In [25]:
if 'fl_date' in df.columns:
    df['year'] = df['year'].astype('Int64') if 'year' in df else df['fl_date'].dt.year.astype('Int64')
    df['month'] = df['month'].astype('Int64') if 'month' in df else df['fl_date'].dt.month.astype('Int64')
else:
    print('Warnung: fl_date nicht gefunden, Monat/Jahr nicht berechnet.')

##  Visualisierungen

In [33]:
# 1) Flüge pro Monat
by_month = df.groupby('month').size().reindex(range(1,13), fill_value=0)

fig, ax = plt.subplots()                       # eigene Figure/Axes
by_month.plot(kind='bar', ax=ax)               # explizit in ax plotten
ax.set_title('Flüge pro Monat')
ax.set_xlabel('Monat'); ax.set_ylabel('Anzahl Flüge')
fig.tight_layout()
fig.savefig("visuals/flights_by_month.png", dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig)       

In [34]:
# 2) Durchschnittliche Ankunftsverspätung nach Airline
delay_airline = df.groupby('op_unique_carrier')['arr_delay'].mean().sort_values(ascending=False)

fig, ax = plt.subplots()
delay_airline.plot(kind='bar', ax=ax)
ax.set_title('Ø Ankunftsverspätung je Airline (Min)')
ax.set_xlabel('Airline'); ax.set_ylabel('Ø Verspätung (Min)')
fig.tight_layout()
fig.savefig("visuals/avg_arr_delay_by_airline.png", dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig)

In [35]:
# 3) Stornorate pro Monat (%)
cancel_rate = (df.groupby('month')['cancelled'].mean()*100).reindex(range(1,13), fill_value=0)

fig, ax = plt.subplots()
cancel_rate.plot(kind='line', marker='o', ax=ax)
ax.set_title('Stornorate pro Monat (%)')
ax.set_xlabel('Monat'); ax.set_ylabel('Stornorate (%)')
fig.tight_layout()
fig.savefig("visuals/cancel_rate_by_month.png", dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig)

In [36]:
# 4) Distanz vs. Flugzeit 
sample = df.sample(min(5000, len(df)), random_state=42)

fig, ax = plt.subplots()
ax.scatter(sample['distance'], sample['air_time'], s=6)
ax.set_title('Distanz vs. Flugzeit (Stichprobe)')
ax.set_xlabel('Distanz (Meilen)'); ax.set_ylabel('Flugzeit (Minuten)')
fig.tight_layout()
fig.savefig("visuals/distance_vs_airtime.png", dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig)

##  Top-Routen (nach Anzahl Flüge)

In [21]:
if {'origin_city_name','dest_city_name'}.issubset(df.columns):
    routes = (df
              .assign(route = df['origin_city_name'] + ' → ' + df['dest_city_name'])
              .groupby('route').size().sort_values(ascending=False).head(20)
             )
    display(routes.to_frame('flights'))
else:
    print('Spalten origin_city_name bzw. dest_city_name fehlen.')

,flights
route,
"Los Angeles, CA → New York, NY",24
"New York, NY → Chicago, IL",21
"Kahului, HI → Honolulu, HI",21
"New York, NY → Los Angeles, CA",21
"Los Angeles, CA → Las Vegas, NV",20
"Honolulu, HI → Kahului, HI",20
"Atlanta, GA → Washington, DC",20
"Chicago, IL → New York, NY",19
"New York, NY → Charlotte, NC",19
